# Filtro basado en contenido

Este tipo de filtro no involucra a otros usuarios si no que está basado en uno mismo (recomendación por contenido, no por opiniones subjetivas de otros usuarios). Según lo que le guste al usuario, el algoritmo simplemente seleccionará elementos con contenido similar para recomendarnos.
En este caso, habrá menos diversidad en las recomendaciones, pero esto funcionará si el usuario califica las cosas o no. 

Un ejemplo de la poca diversidad puede ser el siguiente: tal vez al usuario A le guste potencialmente la comedia negra, pero nunca lo sabrá, a menos que decida intentarlo de forma autónoma, porque este filtro solo seguirá recomendando películas distópicas o similares a las vistas o valoradas anteriormente (historial).

## Dataset

Se ha escogido un dataset de IMDB con las 250 peliculas más valoradas para la implementación del filtro basado en contenido.

In [ ]:
!pip install rake_nltk
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer


#Conjunto de datos que contiene las 250 películas mejor calificadas de IMDB
df = pd.read_csv('https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7')

df.head()


,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


### Limpieza del dataset

Viendo que cada fila (película) contiene 38 columnas o características, se han decidico coger las que pueden ser más interesantes o representativas sobre una película:

1.   Titulo.
2.   Genero.
3.   Director.
4.   Actores.
5.   Trama o Resumen de la película.


In [ ]:
#Coger las columnas de director, genero, director, actores y trama de cada pelicula
df = df[['Title','Genre','Director','Actors','Plot']]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


Se ha utilizado la función ***Rake*** implementada en el paquete ***nltk*** de python para extraer palabras clave de la columna "Trama" ("Plot") de las películas, por lo que en lugar de utilizar las frases completas que describen la trama, solo se considerarán las palabras más relevantes en la descripción.

Aparte de eso, las otras columnas también necesitan limpieza. Es recomendable que todas las palabras estén en minúsculas para evitar duplicaciones, y también se ha decicido combinar todos los nombres y apellidos en una sola palabra. Porque, podría ocurrir lo siguiente: si tenemos la película A donde el director es Danny Boyle, y la película B donde uno de los actores principales es Danny DeVito, el recomendador detectará una similitud por su primer nombre y eso es algo que perjudicaría la calidad de recomendación.

Es mejor que el recomendador detecte una similitud solo si la persona asociada a diferentes películas es exactamente la misma, y no solo por tener el mismo nombre.

In [ ]:

#Columna nueva con las palabras clave
df['Palabras_clave'] = ""

#Iterar sobre todas las peliculas
for index, row in df.iterrows():
    #Seleccionar la columna de trama de cada pelicula
    trama = row['Plot']
    
    #Poner todas las palabras en minusculas y juntarlas (preprocesado para evitar duplicaciones)
    #row['Title']=row['Title'].lower().replace(' ', '')
    row['Genre']=row['Genre'].lower()
    row['Director']=row['Director'].lower().replace(' ', '')
    row['Actors']=row['Actors'].lower().replace(' ', '')


    #Instanciar Rake, por defecto en ingles de NLTK, y descarta la puntuacion
    r = Rake()

    #Extraer palabras pasando el texto
    r.extract_keywords_from_text(trama)

    #Obtener diccionario con las palabras clave y las puntuaciones
    dict_scores = r.get_word_degrees()
    
    #Asignar las palabras clave a la nueva columna para la pelicula correspondiente
    row['Palabras_clave'] = list(dict_scores.keys())

#Eliminar columna plot
df.drop(columns = ['Plot'], inplace = True)

#Poner las palabras de las columnas genero y actores por separado
df['Genre'] = df['Genre'].map(lambda x: x.split(','))
df['Actors'] = df['Actors'].map(lambda x: x.split(','))

#Poner el titulo de la pelicula como columna indice del dataframe
df.set_index('Title', inplace = True)
df.head()


,Genre,Director,Actors,Palabras_clave
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton, william...","[finding, solace, years, acts, eventual, redem..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan, richards.c...","[reluctant, son, aging, patriarch, clandestine..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton, robertde...","[family, crime, syndicate, portrayed, vito, co..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart, mic...","[ability, gotham, physical, tests, mysterious,..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb, e.g.mar...","[justice, prevent, forcing, jury, holdout, att..."


Una vez hecha la limpieza de todas las columnas, se han metido todas las palabras de las columnas (quitando la del título de la pelicula, la cual se utilizará como índice del dataframe) en una única columna llamada "saco de palabras", para realizar un conteo más sencillo.

In [ ]:
#Definir columna vacia
df['saco_palabras']=''
columnas=df.columns

#Correr todo el dataframe
for index,row in df.iterrows():
  palabras=''
  for col in columnas:
    #Si la columna no es la de director, juntar las palabras sueltas de la tupla con separacion entre ellas
    if col!='Director':
      palabras = palabras + ' '.join(row[col])+ ' '
    else:
      #Sino (director), juntar normal
      palabras=palabras + row[col]+ ' '
  #Meter todas las palabras en la columna de saco de palabras
  row['saco_palabras'] = palabras

#Quitar todas las columnas menos la de saco de palabras
df.drop(columns = [col for col in df.columns if col!= 'saco_palabras'], inplace = True)

df.head()

,saco_palabras
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...


Para detectar similitudes entre películas, se necesita vectorizar los datos para calcular la similitud entre estos. Se ha decicido usar la función ***CountVectorizer*** para crear un contador de frecuencia simple para cada palabra en la columna "Saco de palabras".


In [ ]:
#Instanciar y generar matriz de conteo con la columna de saco de palabras
count = CountVectorizer()
count_matrix=count.fit_transform(df['saco_palabras'])

#Crear una serie para los titulos de las peliculas, para tener un orden numerico
#(utilizado para hacer un matching the los indices a la hora de la recomendación)
indices = pd.Series(df.index)
indices[:5]

0    The Shawshank Redemption
1               The Godfather
2      The Godfather: Part II
3             The Dark Knight
4                12 Angry Men
Name: Title, dtype: object

Una vez que se tiene la matriz que contiene el recuento de cada palabra, se puede aplicar la función de similadirad del coseno mediante la función ***cosine_similarity***.

In [ ]:
#Matriz de similitud de coseno
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.14638501, 0.1315587 , ..., 0.05      , 0.05      ,
        0.05270463],
       [0.14638501, 1.        , 0.34236839, ..., 0.048795  , 0.048795  ,
        0.05143445],
       [0.1315587 , 0.34236839, 1.        , ..., 0.0438529 , 0.0438529 ,
        0.04622502],
       ...,
       [0.05      , 0.048795  , 0.0438529 , ..., 1.        , 0.05      ,
        0.05270463],
       [0.05      , 0.048795  , 0.0438529 , ..., 0.05      , 1.        ,
        0.05270463],
       [0.05270463, 0.05143445, 0.04622502, ..., 0.05270463, 0.05270463,
        1.        ]])

## Creación de la función de recomendación

Para la función de recomendación, se ha decicido que la función tome el título de una película como entrada y devuelva las 10 películas más similares (valor de similaridad más alto).  

Para ello, una vez que se recibe el titulo como parametro, se buscan los 10 valores más altos (descartando el valor unitario, para que la función no devuelva el mismo título de la película que se introdujo) dentro de la fila correspondiente a la película ingresada, se obtienen los índices correspondientes y se emparejan con la serie de títulos de películas, para devolver la lista de películas recomendadas. 

In [ ]:
#funcion que recive un titulo de pelicula y devuelte el top10 de peliculas recomendadas 
def recomendacion(titulo, cosine_sim=cosine_sim):
  pelis_recomendadas=[]

  #Coger el indice de la pelicula pasada como parametro
  indice= indices[indices==titulo].index[0]

  #crear una serie con las puntuaciones de similitud en orden descendente
  puntuacion_serie=pd.Series(cosine_sim[indice]).sort_values(ascending = False)

  #coger indices de las 10 peliculas mas similares, coger valores del indice 1 en adelante, indice 0 es el propio titulo con similitud 1
  top_10_indices = list(puntuacion_serie.iloc[1:11].index)
    
  #poblar la lista con los titulos de las 10 peliculas mas similares
  for i in top_10_indices:
      pelis_recomendadas.append(list(df.index)[i])
        
  return pelis_recomendadas

###Ejemplo de llamada de la función de recomendación

In [ ]:
#Llamar a la funcion de recomendacion
recomendacion('The Godfather')

['The Godfather: Part II',
 'Scarface',
 'Fargo',
 'Rope',
 'On the Waterfront',
 'Goodfellas',
 'Baby Driver',
 'Cool Hand Luke',
 'Casino',
 'A Clockwork Orange']

Observando los resultados, parece que se obtiene una bastante decente recomendación, como es la segunda parte de ***El Padrino***, o peliculas como ***Fargo*** o ***Scarface*** las cuales tienen el mismo tipo de género que la pelicula ***El Padrino*** (drama, crimen, etc.).